<a href="https://colab.research.google.com/github/GeoffreyKimani/InvetorySystem-VueJS-/blob/master/house_expenditure_anaylsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install PyDrive

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# https://docs.google.com/spreadsheets/d/1EmoHMuC15JxaXORjtYBFe0ybSEIAbrlbmzFVha8_fMs/edit?usp=sharing
downloaded = drive.CreateFile({'id':"1EmoHMuC15JxaXORjtYBFe0ybSEIAbrlbmzFVha8_fMs"})
downloaded.GetContentFile('household_expenditure', mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')

In [5]:
data = pd.read_excel('household_expenditure')

# select only columns of interest
data = data.loc[:, 'Purchaser':'Misc Cost']

# remove rows with no entries for all fields
data.dropna(axis=0, how='all', inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151 entries, 0 to 150
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Purchaser  122 non-null    object 
 1   Date       122 non-null    object 
 2   Cost       122 non-null    float64
 3   Category   122 non-null    object 
 4   Item       122 non-null    object 
 5   Month      89 non-null     object 
 6   Food Cost  151 non-null    float64
 7   Misc Cost  95 non-null     float64
dtypes: float64(3), object(5)
memory usage: 10.6+ KB


In [6]:
# select all entries where names are unique and form their own dataframes
name_list = ['NK', 'MX', 'JO', 'GK']
purchaser_list = {}

for name in name_list:
    new_df = name + '_df'
    new_df = data[data['Purchaser'] == name]
    purchaser_list.update({name: new_df})

purchaser_list

{'GK':     Purchaser                 Date     Cost  ...     Month Food Cost Misc Cost
 0          GK  2021-08-30 00:00:00   1600.0  ...  Aug 2021    1600.0       0.0
 2          GK  2021-09-02 00:00:00  11400.0  ...  Sep 2021       0.0   11400.0
 3          GK  2021-09-02 00:00:00   1400.0  ...  Sep 2021    1400.0       0.0
 5          GK  2021-09-04 00:00:00  12400.0  ...  Sep 2021       0.0   12400.0
 6          GK  2021-09-04 00:00:00   3200.0  ...  Sep 2021    3200.0       0.0
 11         GK  2021-09-12 00:00:00  14000.0  ...  Sep 2021   14000.0       0.0
 13         GK           13/09/2021   3200.0  ...  Sep 2021    3200.0       0.0
 14         GK           15/09/2021  11600.0  ...  Sep 2021   11600.0       0.0
 15         GK           17/09/2021   5300.0  ...  Sep 2021    5300.0       0.0
 19         GK           26/09/2021   9500.0  ...  Sep 2021    9500.0       0.0
 21         GK           30/09/2021   3200.0  ...  Sep 2021    3200.0       0.0
 23         GK           03/10/202

In [7]:
# total expenses
purchaser_expense = {}
for purchaser, df in purchaser_list.items():
    purchaser_expense.update({purchaser: df['Cost'].sum()})

purchaser_expense

{'GK': 218700.0, 'JO': 310430.0, 'MX': 340450.0, 'NK': 121300.0}

#Share Contribution and Deficit

In [8]:
total_expense = sum(purchaser_expense.values())
total_expense

990880.0

In [9]:
# personal share
personal_share = total_expense/len(purchaser_expense)
personal_share

247720.0

In [10]:
# deficit
deficits = {person: contribution - personal_share for person, contribution in purchaser_expense.items()}
deficits

{'GK': -29020.0, 'JO': 62710.0, 'MX': 92730.0, 'NK': -126420.0}

# Monthly Analysis 💰

In [11]:
# plt.bar(total_expense.keys(), total_expense.values())